### Scraping t20 batsman ranking

In [2]:
import sys,requests,csv,io
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "http://www.cricbuzz.com/cricket-stats/icc-rankings/batsmen-rankings"
r = requests.get(url)
r.content
soup = BeautifulSoup(r.content, "html.parser")

In [28]:
lis = []
maindiv = soup.select("#batsmen-t20s div.text-center")
for d in maindiv[1:]:
    row_data = u",".join(s.strip() for s in filter(None, (t.find(text=True, recursive=False) for t in d.find_all())))
    if row_data:
        print(row_data)
        lis.append(row_data)

1,,,3,,Colin Munro,NEW ZEALAND,801,1
2,,,5,,Glenn Maxwell,AUSTRALIA,799,2
3,,,2,,Babar Azam,PAKISTAN,786,1
4,,,2,,Aaron Finch,AUSTRALIA,763,1
5,,,6,,Martin Guptill,NEW ZEALAND,747,1
6,,,3,,Virat Kohli,INDIA,741,1
7,,,2,,Evin Lewis,WINDIES,734,2
8,,,2,,Lokesh Rahul,INDIA,683,4
9,,,1,,Alex Hales,ENGLAND,679,1
10,,,2,,Mohammad Shahzad,AFGHANISTAN,653,7
11,,,–,,Hamilton Masakadza,ZIMBABWE,648,4
12,,,2,,Hashim Amla,SOUTH AFRICA,643,8
13,,,–,,Kane Williamson,NEW ZEALAND,634,3
14,,,5,,Joe Root,ENGLAND,630,4
15,,,2,,Eoin Morgan,ENGLAND,618,1
16,,,–,,Shaiman Anwar,UNITED ARAB EMIRATES,615,12
17,,,2,,Rohit Sharma,INDIA,604,11
18,,,4,,Faf du Plessis,SOUTH AFRICA,594,3
19,,,–,,David Warner,AUSTRALIA,589,1
20,,,5,,Soumya Sarkar,BANGLADESH,581,20
21,,,3,,Sabbir Rahman,BANGLADESH,578,10
22,,,–,,Dawid Malan,ENGLAND,576,22
23,,,–,,Malcolm Waller,ZIMBABWE,572,22
24,,,4,,Jean-Paul Duminy,SOUTH AFRICA,568,6
25,,,3,,Marlon Samuels,WINDIES,568,3
26,,,6,,Jos Buttler,ENGLAND,558,18
28,,,22,,Shikhar Dhawan,IND

In [29]:
import pandas as pd
df = pd.DataFrame(lis,columns = ['rankings'])

In [30]:
df['current_rank'] = df['rankings'].apply(lambda x: str(x).split(',')[0])
df['batsman'] = df['rankings'].apply(lambda x: str(x).split(',')[5])
df['rating'] = df['rankings'].apply(lambda x: str(x).split(',')[7])
df['best_rank'] = df['rankings'].apply(lambda x: str(x).split(',')[8])

In [32]:
df.drop('rankings',axis=1,inplace=True)

In [36]:
df.to_csv('/Users/shubhamjain/Downloads/AV/IPL/rating.csv', index=False)

In [37]:
df.head()

,current_rank,batsman,rating,best_rank
0,1,Colin Munro,801,1
1,2,Glenn Maxwell,799,2
2,3,Babar Azam,786,1
3,4,Aaron Finch,763,1
4,5,Martin Guptill,747,1


In [108]:
## auction data

auction = pd.read_csv('/Users/shubhamjain/Downloads/AV/IPL/cricbuzz_ipl.csv')

In [109]:
auction['country'] = auction['role'].apply(lambda x: str(x).split()[-1])

In [116]:
auction['Price'] = auction['final_price'].apply(lambda x: str(x).split()[0])
auction['final_price'] = auction['final_price'].apply(lambda x: str(x).split()[1])

In [117]:
auction = auction.loc[auction['Price'] != '-']

In [127]:
auction.loc[auction['final_price'] == 'Crore', 'Price'] = auction.loc[auction['final_price'] == 'Crore', 'Price'].astype('float') * 100

In [133]:
df = auction[['player','Price','country']]

In [135]:
df.head()

,player,Price,country
0,Ben Stokes,1250,England
1,Jaydev Unadkat,1150,India
2,Manish Pandey,1100,India
3,Lokesh Rahul,1100,India
4,Chris Lynn,960,Australia


In [139]:
df['country'].value_counts()

India          113
Australia       17
England          8
Africa           8
Windies          7
Zealand          7
Afghanistan      4
Lanka            2
Bangladesh       2
Nepal            1
Name: country, dtype: int64

In [145]:
retained = pd.DataFrame(data = {
    'player': ['MS Dhoni','Suresh Raina','Ravindra Jadeja', 'Rishabh Pant', 'Chris Morris', 'Shreyas Iyer',
              'Axar Patel', 'Sunil Narine', 'Andre Russell', 'Rohit Sharma', 'Hardik Pandya', 'Jasprit Bumrah',
              'Steve Smith', 'Virat Kohli', 'AB de Villiers', 'Sarfaraz Khan', 'David Warner', 'Bhuvneshwar Kumar'],
    'Price': [15,11,7,8,7.1,7,6.75,8.5,7,15,11,7,12,17,11,1.75,12,8.5] ,
    'country':['India','India','India','India','Africa','India','India','Windies', 'Windies','India','India','India',
              'Australia','India','Africa','India','Australia','India']
 })

In [149]:
retained['Price'] = retained['Price'] * 100

In [152]:
df = pd.concat([df,retained]).reset_index(drop=True)

In [175]:
df['player'][0]

'Ben Stokes'

In [173]:
df.ix[:168,'player'] = df.ix[:168,'player'].apply(lambda x: str(x).split(" ")[1] +" "+ str(x).split(" ")[2])

In [174]:
df.to_csv('/Users/shubhamjain/Downloads/AV/IPL/cricbuzz_ipl.csv', index=False)